In [1]:
import zarr # Check version
zarr.__version__

'2.13.2'

Zarr Sharding pull request should show '0.0.0'

If it says e.g. 2.13.2, then you're running on the "proper" published version (without Sharding support as of 28-Sept-2022)

### 1. I wanna test partial_decompression by accessing a random index

In [5]:
import numpy as np


numpy_arr = np.empty((50000, 15000), dtype=np.uint32) # ~3GB
original_shape = numpy_arr.shape

# 50k x 150k int32 is limit of M1 Pro 32GB
#
for i in range(numpy_arr.shape[0]):
    numpy_arr[i][:] = list(range(original_shape[1]*i, original_shape[1]*(i+1)))

In [13]:
# Let's make chunks really large to test partial decompression
# 10k x 10k ~ 800MB per chunk

zarr_index_list = zarr.array(numpy_arr, chunks=(10000,10000), partial_decompress=True, dtype='f8')
# z = zarr.zeros((10000,10000), chunks=(1000,1000), dtype='i4')
# z

In [16]:
# example-zarr is actually a folder
z1 = zarr.open('./example-zarr', mode='w', shape=(10000,10000), chunks=(1000,1000), dtype='i4')
# I don't think it's flushed to disk yet. Only folder created

In [17]:
!ls

example-zarr         tutorials            zarr_python
example.zarr         zarr-explore-1.ipynb


In [5]:
# # Read from array
# z2 = zarr.open("./example.zarr", mode="r")
# z2

<zarr.core.Array (10000, 10000) int32 read-only>

In [ ]:
# # Check if reading from file == original array
# import numpy as np

# np.all(z[:] == z2[:])

#### <font color="orange">Trying Compressors</font>

In [18]:
# Blosc is one of the preconditions of partial_decompress
# https://zarr.readthedocs.io/en/stable/api/core.html
from numcodecs import Blosc

bloccc = Blosc(clevel=9)

In [19]:
z_compressed = zarr.array(zarr_index_list, compressor = bloccc)

In [20]:
z_compressed.compressor

Blosc(cname='lz4', clevel=9, shuffle=SHUFFLE, blocksize=0)

In [21]:
z_compressed

<zarr.core.Array (10000, 10000) int64>

In [22]:
zarr.save('./example.zarr', z_compressed)

In [15]:
z2



KeyboardInterrupt

